# Credit
Original Notebook https://www.kaggle.com/code/rsuhara/ai-generated-text-detection-quick-baseline

Double Fork from [AI Generated Text Detection - add new data | Kaggle](https://www.kaggle.com/code/xiaocao123/ai-generated-text-detection-add-new-data)

Inspired by VLADIMIR DEMIDOV's work : <br>
https://www.kaggle.com/code/yekenot/llm-detect-by-regression

For the training data we shall use the "RDizzl3 seven" dataset (v1) which can be found in the "LLM: 7 prompt training dataset" https://www.kaggle.com/datasets/carlmcbrideellis/llm-7-prompt-training-dataset



# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier

# Data Consolidation and Feature Engineering

In [ ]:
train_datas=pd.read_csv("/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv")

In [ ]:
trains_data=train_datas[train_datas["label"]==1]
trains_data=trains_data[["text","label"]]
trains_data['text'] = trains_data['text'].str.replace('\n', '')
trains_data.head(3)

In [ ]:
external_df = pd.read_csv("/kaggle/input/daigt-external-dataset/daigt_external_dataset.csv", sep=',')
print(external_df.shape)
external_df = external_df.rename(columns={'generated': 'label'})
external_df = external_df[["source_text"]]
external_df.columns = ["text"]
external_df['text'] = external_df['text'].str.replace('\n', '')
external_df["label"] = 1

In [ ]:
train = pd.read_csv("/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v1.csv")
train=pd.concat([train,external_df,trains_data])
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

df = pd.concat([train['text'], test['text']], axis=0)

vectorizer = TfidfVectorizer(ngram_range=(1, 3),sublinear_tf=True)
X = vectorizer.fit_transform(df)

In [ ]:
train.value_counts("label")

# Logistic Regression

In [ ]:
lr_model = LogisticRegression()

# SGD

In [ ]:
sgd_model = SGDClassifier(max_iter=5000, tol=1e-3, loss="modified_huber")

# XGBoot

In [ ]:
# xgboost=XGBClassifier()

# SVC

In [ ]:
# from sklearn.svm import SVC
# svc_model = SVC(probability=True)

# KNN

In [ ]:
# clf = KNeighborsClassifier()

# Voting Classifier

In [ ]:
# Create the ensemble model
ensemble = VotingClassifier(estimators=[('lr', lr_model),('sgd', sgd_model)],weights=[0.1,0.9],voting='soft')
ensemble.fit(X[:train.shape[0]], train.label)

In [ ]:
preds_test = ensemble.predict_proba(X[train.shape[0]:])[:,1]

In [ ]:
pd.DataFrame({'id':test["id"],'generated':preds_test}).to_csv('submission.csv', index=False)